<a href="https://colab.research.google.com/github/bmnds/uea-data-science-04-data-processing/blob/main/%5BAtividade02%5D_Notebook_ETL_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando libs necessárias
import pandas as pd
import numpy as np

In [2]:
# Baixando datasets para análise
!curl -o brazil_covid19.tar.gz -s -OL 'https://drive.google.com/uc?export=download&id=1gsOhZkvDKCiEwtjwJbcyyXdbhjyOIukn'
!tar -zxvf brazil_covid19.tar.gz
!rm brazil_covid19.tar.gz

brazil_covid19/
brazil_covid19/brazil_cities_coordinates.csv
brazil_covid19/brazil_covid19.csv
brazil_covid19/brazil_covid19_cities.csv
brazil_covid19/brazil_covid19_macro.csv
brazil_covid19/brazil_population_2019.xlsx


# JUNÇÃO E CLASSIFICAÇÃO
*   Ler os dados de diferentes tabelas;
*   Extrair variável de interesse;
*   Aplicar transformações.



In [3]:
df_pop = pd.read_excel('brazil_covid19/brazil_population_2019.xlsx')
df_pop.sample(5) 

,region,state,city,state_code,city_code,health_region_code,health_region,population
986,Nordeste,Alagoas,Taquarana,27,270910,27007,7ª Região de Saúde,19980
2379,Sul,Paraná,Campo Bonito,41,410405,41010,10ª RS Cascavel,3833
851,Nordeste,Pernambuco,Lagoa do Carro,26,260845,26006,Limoeiro,18071
214,Nordeste,Maranhão,Guimarães,21,210490,21011,Pinheiro,12030
1500,Norte,Tocantins,Augustinópolis,17,170255,17002,Bico do Papagaio,18412


In [4]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2865 entries, 0 to 2864
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   region              2865 non-null   object
 1   state               2865 non-null   object
 2   city                2865 non-null   object
 3   state_code          2865 non-null   int64 
 4   city_code           2865 non-null   int64 
 5   health_region_code  2865 non-null   int64 
 6   health_region       2865 non-null   object
 7   population          2865 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 179.2+ KB


In [5]:
regions = np.unique(df_pop['region'])
regions

array(['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul'],
      dtype=object)

In [6]:
all_regions_pop = [ df_pop['population'].loc[df_pop['region'] == region].sum() for region in regions ]
all_regions_pop

regions_pop = [('Centro-Oeste',all_regions_pop[0]),('Nordeste',all_regions_pop[1]),\
                      ('Norte',all_regions_pop[2]),('Sudeste',all_regions_pop[3]),\
                      ('Sul',all_regions_pop[4])]

regions_population = pd.DataFrame.from_dict(dict(regions_pop),orient='index',columns=['population'])
regions_population

,population
Centro-Oeste,13085364
Nordeste,48946190
Norte,17328833
Sudeste,80421777
Sul,24522343


In [7]:
df_cases = pd.read_csv('brazil_covid19/brazil_covid19.csv',error_bad_lines=False)
df_cases.sample(5)

,date,region,state,cases,deaths
3614,2020-07-07,Sudeste,SP,332708,16475
4019,2020-07-22,Sudeste,SP,439446,20532
6028,2020-10-05,Nordeste,MA,175753,3807
4286,2020-08-01,Sudeste,ES,83814,2566
4768,2020-08-19,Norte,PA,182231,6015


In [8]:
"""
Encontrando total de óbitos por região
"""
all_regions_deaths = [ df_cases['deaths'].loc[df_cases["region"]== region].sum() for region in regions ]
all_regions_deaths

regions_deaths = [('Centro-Oeste',all_regions_deaths[0]),('Nordeste',all_regions_deaths[1]),\
                      ('Norte',all_regions_deaths[2]),('Sudeste',all_regions_deaths[3]),\
                      ('Sul',all_regions_deaths[4])]

regions_deaths = pd.DataFrame.from_dict(dict(regions_deaths),orient='index',columns=['deaths'])
regions_deaths

,deaths
Centro-Oeste,1518353
Nordeste,5812513
Norte,2386633
Sudeste,9326839
Sul,1464191


In [9]:
"""
Encontrando total de casos por região
"""

all_regions_cases = [ df_cases['cases'].loc[df_cases["region"]== region].sum() for region in regions ]

regions_cases = [('Centro-Oeste',all_regions_cases[0]),('Nordeste',all_regions_cases[1]),\
                      ('Norte',all_regions_cases[2]),('Sudeste',all_regions_cases[3]),\
                      ('Sul',all_regions_cases[4])]

regions_cases = pd.DataFrame.from_dict(dict(regions_cases),orient='index',columns=['cases'])
regions_cases

,cases
Centro-Oeste,71590231
Nordeste,184449738
Norte,89017773
Sudeste,224057624
Sul,72207079


## JUNÇÃO

> Realiza-se a junção do arquivo *brazil_population_2019.csv*, contendo a informação populacional com *brazil_covid19.csv*, a qual informa o número de casos e óbitos, a fim de posteriormente aplicar classificação.

In [10]:
df = pd.concat([regions_population,regions_cases],axis=1)
df = pd.concat([df,regions_deaths],axis=1)
df

,population,cases,deaths
Centro-Oeste,13085364,71590231,1518353
Nordeste,48946190,184449738,5812513
Norte,17328833,89017773,2386633
Sudeste,80421777,224057624,9326839
Sul,24522343,72207079,1464191


## CLASSIFICAÇÃO

* Classificar porcentagem de óbitos por região.
* Classificar porcentagem de casos de cura.
* Obs.: Dados referentes ao período de **27/03** a **25/11** de 2020


* A coluna ***deaths_percent*** representa a porcentagem de óbitos da população.
* A coluna *safe_cases* representa a porcentagem de casos detectados e curados.

In [11]:
"""
Ordenando por porcentagem de óbitos
"""

df['deaths_percent'] = (df['deaths']/df['population'])*100
df['safe_cases'] = ((df['cases']-df['deaths'])/df['cases'])*100
df.sort_values('deaths_percent',ascending=False)

,population,cases,deaths,deaths_percent,safe_cases
Norte,17328833,89017773,2386633,13.772612,97.318925
Nordeste,48946190,184449738,5812513,11.875312,96.848728
Centro-Oeste,13085364,71590231,1518353,11.603445,97.879106
Sudeste,80421777,224057624,9326839,11.597405,95.837303
Sul,24522343,72207079,1464191,5.970845,97.972233


In [12]:
"""
Ordenando por casos detectados e curados.
"""
df.sort_values('safe_cases',ascending=False)

,population,cases,deaths,deaths_percent,safe_cases
Sul,24522343,72207079,1464191,5.970845,97.972233
Centro-Oeste,13085364,71590231,1518353,11.603445,97.879106
Norte,17328833,89017773,2386633,13.772612,97.318925
Nordeste,48946190,184449738,5812513,11.875312,96.848728
Sudeste,80421777,224057624,9326839,11.597405,95.837303


# OPERAÇÃO DE LINHA E AGREGAÇÃO

Para a análise de operações de linha e agregação, utilizaremos o dataset **brazil_covid19_cities.csv** que contém a quantidade acumulada de casos confirmados e de óbitos por covid19, dia a dia, para cada município do Brasil

In [13]:
df_cities = pd.read_csv('brazil_covid19/brazil_covid19_cities.csv')
df_cities.sample(5)

,date,state,name,code,cases,deaths
328664,2020-05-25,AL,Cajueiro,270130.0,7,1
250163,2020-05-10,SP,Lagoinha,352630.0,0,0
145121,2020-04-22,BA,Cocos,290810.0,0,0
1299793,2020-11-15,MG,Rio Acima,315480.0,287,4
991917,2020-09-21,BA,Miguel Calmon,292120.0,83,4


1. *Operação de linha*: Converteremos os valores da coluna **date** para o tipo **datetime**;
2. *Agregação*: Calcularemos o somatório de casos confirmados e de óbitos **por data e estado**;
3. *Operação de linha*: Criaremos uma nova coluna que representará a taxa de óbitos em relação à quantidade de casos confirmados;
4. *Classificação*: Verificaremos quais são os 5 estados com maior taxa de óbitos na última data da nossa amostragem, em 25 de novembro de 2020
* OBS: optamos por analisar a nível de Estado ao invés de Cidade para reduzir o volume de dados processados

### Operação de linha

In [14]:
# 1. Operação de linha: Converteremos os valores da coluna date para o tipo datetime;
df_cities.date = pd.to_datetime(df_cities.date)
print(f'Tipos de dados após a operação:')
df_cities.dtypes

Tipos de dados após a operação:


date      datetime64[ns]
state             object
name              object
code             float64
cases              int64
deaths             int64
dtype: object

Podemos constatar que os dados são referentes ao período de  **25/02** a **25/11** de 2020

In [15]:
df_cities.date.describe(datetime_is_numeric=True)

count                          1359080
mean     2020-07-26 11:59:59.999321856
min                2020-03-27 00:00:00
25%                2020-05-26 18:00:00
50%                2020-07-26 12:00:00
75%                2020-09-25 06:00:00
max                2020-11-25 00:00:00
Name: date, dtype: object

### Agregação

In [16]:
# 2. Agregação: Calcularemos o somatório de casos confirmados e de óbitos por data e estado;
df_states = df_cities.groupby(['date','state'])[['cases','deaths']].sum().reset_index()
df_states.sample(5)

,date,state,cases,deaths
1081,2020-05-06,AL,1692,87
4633,2020-09-14,PI,86538,1973
5416,2020-10-13,PI,102813,2237
5445,2020-10-14,RJ,285205,19440
1541,2020-05-23,AM,28802,1744


### Análise

In [17]:
# 3. Operação de linha: Criaremos uma nova coluna que representará a taxa de óbitos em relação à quantidade de casos confirmados;
df_states['taxa_obitos'] = df_states.apply(
    lambda row: 0 if row['cases'] == 0 else 100*row['deaths']/row['cases'], 
    axis=1)

In [18]:
# 4. Classificação: Verificaremos quais são os 5 estados com maior taxa de óbitos na última data da nossa amostragem, em 25 de novembro de 2020
df_states[df_states['date'].eq(df_states.date.max())].sort_values('taxa_obitos',ascending = False).head(5)

,date,state,cases,deaths,taxa_obitos
6579,2020-11-25,RJ,343995,22256,6.469861
6576,2020-11-25,PE,178086,8971,5.037454
6586,2020-11-25,SP,1224544,41601,3.397265
6566,2020-11-25,CE,290887,9530,3.276186
6580,2020-11-25,RN,88064,2665,3.026208
